In [160]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer
dev = qml.device("default.qubit")
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from pennylane.optimize import NesterovMomentumOptimizer

In [161]:
num_qubits = 4
num_layers = 6
@qml.qnode(dev)
def circuit(x, weights):
    # Angle embedding
    # print("Input:",x)
    qml.templates.AngleEmbedding(x, wires=range(num_qubits))

    # Basic entangler layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))

    # qml.CNOT(wires=[10,0])
    # Measurement
    return qml.expval(qml.PauliX(0))


In [162]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc


In [163]:
def cost(weights, X, Y):
    predictions = circuit(X, weights)
    
    # Sigmoid activation to convert to probabilities
    predictions = 1 / (1 + np.exp(-predictions,requires_grad = True))

    # Calculate BCE loss
    bce_loss = -np.mean(Y * np.log(predictions + 1e-10) + (1 - Y) * np.log(1 - predictions + 1e-10))
    return bce_loss

In [164]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# Load the dataset (adjust the path to your file)
df = pd.read_csv('credit_risk_dataset.csv')

# Display the first few rows


# Check for missing values
print("Missing values in each column:\n", df.isnull().sum())

# Preprocessing: Fill or drop missing values as necessary
# df.fillna(0, inplace=True)

df.dropna(inplace=True)
df = df.reset_index(drop=True)
df = df.select_dtypes(exclude=['object'])
# Identify categorical columns
# categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Initialize LabelEncoder
# label_encoder = LabelEncoder()

# Encode categorical columns
# for col in categorical_cols:
    # df[col] = label_encoder.fit_transform(df[col])

# Define features and target variable
X = df.drop('loan_status', axis=1)  # Replace 'loan_status' with your target variable name
# X = X.drop(['person_emp_length', 'person_age'],axis=1)
X = X[['person_age','person_income','person_emp_length','loan_amnt']]
scaler = MinMaxScaler()
X['loan_amnt'] = scaler.fit_transform(X[['loan_amnt']])
y = df['loan_status']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test= train_test_split(X_test, y_test, test_size=0.5, random_state=42)
# print(X_train.iloc[0])

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_train[y_train==0] = -1
y_test[y_test==0] = -1
y_val[y_val==0] = -1
# print("Shapes of the training and test sets:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


Missing values in each column:
 person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64
X_train: (22910, 4), y_train: (22910,)
X_test: (2864, 4), y_test: (2864,)


In [175]:
np.random.seed(0)
weights_init = np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
# bias_init = np.array(0.0, requires_grad=True)
#
# print("Weights:", weights_init)
# print("Bias: ", bias_init)

In [176]:
circuit_draw = qml.draw(circuit,level="device")(X_train[0], weights_init)  # Visualize with the first training sample
print(circuit_draw)
print(X_train[0])

0: ──RX(-0.43)──Rot(1.76,0.40,0.98)───╭●───────╭X──Rot(0.76,0.12,0.44)───╭●────╭X
1: ──RX(0.59)───Rot(2.24,1.87,-0.98)──╰X─╭●────│───Rot(0.33,1.49,-0.21)──│──╭●─│─
2: ──RX(-0.67)──Rot(0.95,-0.15,-0.10)────╰X─╭●─│───Rot(0.31,-0.85,-2.55)─╰X─│──╰●
3: ──RX(-0.01)──Rot(0.41,0.14,1.45)─────────╰X─╰●──Rot(0.65,0.86,-0.74)─────╰X───

───Rot(2.27,-1.45,0.05)────────────────────────╭●─╭X──Rot(1.23,1.20,-0.39)────────────────────────
──╭X─────────────────────Rot(-0.19,1.53,1.47)──│──╰●─╭X─────────────────────Rot(-0.30,-1.05,-1.42)
──│──────────────────────Rot(0.15,0.38,-0.89)──│─────╰●────────────────────╭X─────────────────────
──╰●─────────────────────Rot(-1.98,-0.35,0.16)─╰X──────────────────────────╰●─────────────────────

──╭●───────────────────────────╭X──Rot(-1.61,-0.21,-0.90)─╭●────╭X──Rot(-0.67,-0.36,-0.81)
──╰X─────────────────────╭●────│───Rot(0.39,-0.51,-1.18)──│──╭●─│──╭X─────────────────────
───Rot(-1.71,1.95,-0.51)─╰X─╭●─│───Rot(-0.03,0.43,0.07)───╰X─│──╰●─│──────────────────────
─

In [177]:
weights = weights_init
# bias = bias_init
opt = NesterovMomentumOptimizer(0.01)
batch_size = 32
costs = []
for it in range(1000):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    batch_index_val = np.random.randint(0, len(X_val), (batch_size,))
    X_batch = X_train[batch_index]
    # print(X_batch)
    Y_batch = y_train[batch_index]

    X_val_b = X_val[ batch_index_val]
    Y_val_b = y_val[ batch_index_val]
    
    # print(Y_batch)
    weights= opt.step(cost, weights,  X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(circuit(x,  weights)) for x in X_batch]
    # print(predictions)
    # print(Y_batch)
    current_cost = cost(weights,  X_batch, Y_batch)
    costs.append(current_cost)
    acc = accuracy(Y_batch, predictions)
    
    predictions = [np.sign(circuit(x,  weights)) for x in X_val_b]
    val_acc =accuracy(Y_val_b, predictions)

    if it%10 == 0:
        print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f} | Val_Accuracy: {val_acc:0.7f}")

Iter:    1 | Cost: 0.6350787 | Accuracy: 0.5625000 | Val_Accuracy: 0.4687500
Iter:   11 | Cost: 0.5262793 | Accuracy: 0.5937500 | Val_Accuracy: 0.7500000
Iter:   21 | Cost: 0.2812967 | Accuracy: 0.6875000 | Val_Accuracy: 0.7500000
Iter:   31 | Cost: 0.0303416 | Accuracy: 0.8437500 | Val_Accuracy: 0.7812500
Iter:   41 | Cost: 0.2677883 | Accuracy: 0.6250000 | Val_Accuracy: 0.9687500
Iter:   51 | Cost: 0.0250283 | Accuracy: 0.8125000 | Val_Accuracy: 0.7500000
Iter:   61 | Cost: 0.0993236 | Accuracy: 0.7500000 | Val_Accuracy: 0.7500000
Iter:   71 | Cost: 0.0890100 | Accuracy: 0.7187500 | Val_Accuracy: 0.6250000
Iter:   81 | Cost: 0.0104861 | Accuracy: 0.8125000 | Val_Accuracy: 0.7812500
Iter:   91 | Cost: 0.2361662 | Accuracy: 0.6562500 | Val_Accuracy: 0.7812500
Iter:  101 | Cost: -0.1551433 | Accuracy: 0.9062500 | Val_Accuracy: 0.7812500
Iter:  111 | Cost: -0.0101629 | Accuracy: 0.7500000 | Val_Accuracy: 0.6562500
Iter:  121 | Cost: 0.1694456 | Accuracy: 0.7187500 | Val_Accuracy: 0.71875

In [178]:
predictions = [np.sign(circuit(x,  weights)) for x in X_test]
acc = accuracy(y_test, predictions)
report = classification_report(y_test, predictions)

In [179]:
print(acc)
print(report)

0.772695530726257
              precision    recall  f1-score   support

          -1       0.78      0.99      0.87      2238
           1       0.16      0.01      0.02       626

    accuracy                           0.77      2864
   macro avg       0.47      0.50      0.44      2864
weighted avg       0.65      0.77      0.68      2864



In [55]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Make predictions
y_pred = model.predict(X_test)

# Step 7: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

Accuracy: 0.79
Classification Report:
               precision    recall  f1-score   support

          -1       0.79      0.99      0.88      2238
           1       0.62      0.08      0.14       626

    accuracy                           0.79      2864
   macro avg       0.71      0.53      0.51      2864
weighted avg       0.75      0.79      0.72      2864



In [29]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred_dt = dt_model.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))

Decision Tree Accuracy: 0.8083100558659218
